In [1]:
from multiprocessing import Pool
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import json
import os

chrome_options = Options()
chrome_options.add_argument('--headless')
baseUrl = "https://www.basketball-reference.com/players/"
css_selector = "ul.alphabet.bullets-inline"

def getLink(driver, index):
    if chr(index+97) != 'x': 
        li = driver.find_element_by_css_selector(css_selector).find_elements_by_tag_name('li')[index]
        return li.find_element_by_tag_name('a')
    return None

def spider(index_range):
    players = []
    index = str(os.getpid())
    driver = webdriver.Chrome(chrome_options=chrome_options)
    driver.get(baseUrl)
    print(index+" start")
    
    for link in index_range:
        a = getLink(driver, link)
        if a is None:
            continue
        a.click()
        print(index+" "+chr(97+link))
        table = driver.find_element_by_id('players')
        tbody = table.find_element_by_tag_name('tbody')
        trs = tbody.find_elements_by_tag_name('tr')
        for tr in trs:
            tds = tr.find_elements_by_tag_name('td')
            player = dict(zip(['From',"To","Pos","Height","Weight","Birth","College"],
                [td.text for td in tds]))
            plink = tr.find_element_by_tag_name('th').find_element_by_tag_name('a')
            player['Player'] = plink.text
            print(index+" "+plink.text)
            player['Link'] = plink.get_attribute("href")
            players.append(player)
        driver.back()
    driver.close()
    return players
   

In [3]:
dataFile = "./data/all_players"
print("Start")
p = Pool(5)
index_ranges = [list(range(i,i+5)) for i in range(0,26,5) if i < 25]
index_ranges[-1].append(25)
result = []
print(index_ranges)
for i in range(5):
    result.append(p.apply_async(spider, args=(index_ranges[i],)))
p.close()
p.join()

Start
[[0, 1, 2, 3, 4], [5, 6, 7, 8, 9], [10, 11, 12, 13, 14], [15, 16, 17, 18, 19], [20, 21, 22, 23, 24, 25]]
2768 start
2769 start
2771 start
2770 start
2767 start


In [ ]:
res = []
for r in result:
    res += r.get()
with open(dataFile,'w') as file:
    json.dump(res,file) 